In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import umap

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


In [2]:
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

In [3]:
df = pd.read_csv('droughts_extended.csv')
df_trainVal_withTar = df.iloc[:-392,:]
df_test_withTar = df.iloc[-392:,:]
df_trainVal = df.iloc[:-392,:-1]
df_test = df.iloc[-392:,:-1]
target_df_trainVal = df.iloc[:-392,-1]
target_df_test = df.iloc[-392:,-1]

# Regression

## PCA

In [15]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample

r2 = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    actual_r2 = compare_methods(trainVal_pca, test_pca, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_pca.columns))
    r2.append([curr_seed,actual_r2])
    print(test_pca.shape)
    

Full aggregate regression train score: 0.8675587573184024, test score: 0.7850091641907997
Aggregate regression train score with FS: 0.8675587573184024, test score: 0.7850091641907997
(392, 18)
Full aggregate regression train score: 0.8216159392382211, test score: 0.7633714791597755
Aggregate regression train score with FS: 0.8216159392382211, test score: 0.7633714791597755
(392, 19)
Full aggregate regression train score: 0.8508353097828716, test score: 0.7609848606090145
Aggregate regression train score with FS: 0.8508353097828716, test score: 0.7609848606090145
(392, 18)
Full aggregate regression train score: 0.850535586951923, test score: 0.722959063643331
Aggregate regression train score with FS: 0.850535586951923, test score: 0.722959063643331
(392, 19)
Full aggregate regression train score: 0.8373561962396265, test score: 0.7358441197745467
Aggregate regression train score with FS: 0.8373561962396265, test score: 0.7358441197745467
(392, 17)


In [60]:
n_pca95 = [18,19,18,19,17]
r2_pca95 = [0.7850091641907997,0.7633714791597755,0.7609848606090145,0.722959063643331,0.7358441197745467]
compute_CI(n_pca95,5)
compute_CI(r2_pca95,5)

18.2 +- 0.6559414608027151
0.7536337374754934 +- 0.019166700154449642


## LPP

In [19]:
from lpproj import LocalityPreservingProjection as lpp

In [61]:
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = lpp(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
    

Full aggregate regression train score: 0.00015955388303245943, test score: -2.3747228845071894e+21
Aggregate regression train score with FS: 0.00015955388303245943, test score: -2.3747228845071894e+21
(392, 1)
Full aggregate regression train score: 0.0005377336812445455, test score: -7.351334499775748e+21
Aggregate regression train score with FS: 0.0005377336812445455, test score: -7.351334499775748e+21
(392, 2)
Full aggregate regression train score: 0.003334365524284677, test score: -6.745117650106194e+22
Aggregate regression train score with FS: 0.003334365524284677, test score: -6.745117650106194e+22
(392, 3)
Full aggregate regression train score: 0.003627792140066588, test score: -7.638978024009149e+22
Aggregate regression train score with FS: 0.003627792140066588, test score: -7.638978024009149e+22
(392, 4)
Full aggregate regression train score: 0.004647992072507545, test score: -7.814573788366132e+22
Aggregate regression train score with FS: 0.004647992072507545, test score: -7.8

Full aggregate regression train score: 0.029202396566997746, test score: -8.192649628230843e+23
Aggregate regression train score with FS: 0.029202396566997746, test score: -8.192649628230843e+23
(392, 41)
Full aggregate regression train score: 0.031216730373353818, test score: -7.792225443343707e+23
Aggregate regression train score with FS: 0.031216730373353818, test score: -7.792225443343707e+23
(392, 42)
Full aggregate regression train score: 0.02807900842041333, test score: -7.140182838868293e+23
Aggregate regression train score with FS: 0.02807900842041333, test score: -7.140182838868293e+23
(392, 43)
Full aggregate regression train score: 0.032148562391221214, test score: -9.01941197181927e+23
Aggregate regression train score with FS: 0.032148562391221214, test score: -9.01941197181927e+23
(392, 44)
Full aggregate regression train score: 0.03427396712783082, test score: -9.22846101215587e+23
Aggregate regression train score with FS: 0.03427396712783082, test score: -9.228461012155

Full aggregate regression train score: 0.029715205088469143, test score: -1.299796235308871e+24
Aggregate regression train score with FS: 0.029715205088469143, test score: -1.299796235308871e+24
(392, 32)
Full aggregate regression train score: 0.03658220474249385, test score: -1.8018286119798734e+24
Aggregate regression train score with FS: 0.03658220474249385, test score: -1.8018286119798734e+24
(392, 33)
Full aggregate regression train score: 0.03865050583019769, test score: -2.0103372525609995e+24
Aggregate regression train score with FS: 0.03865050583019769, test score: -2.0103372525609995e+24
(392, 34)
Full aggregate regression train score: 0.04078399731237048, test score: -2.1609911318088298e+24
Aggregate regression train score with FS: 0.04078399731237048, test score: -2.1609911318088298e+24
(392, 35)
Full aggregate regression train score: 0.04018156148437768, test score: -1.8990388976933685e+24
Aggregate regression train score with FS: 0.04018156148437768, test score: -1.899038

Full aggregate regression train score: 0.015289978709732255, test score: -2.947064742393807e+23
Aggregate regression train score with FS: 0.015289978709732255, test score: -2.947064742393807e+23
(392, 22)
Full aggregate regression train score: 0.015470200895231323, test score: -3.298053873057145e+23
Aggregate regression train score with FS: 0.015470200895231323, test score: -3.298053873057145e+23
(392, 23)
Full aggregate regression train score: 0.01755192821102436, test score: -3.9100953861492224e+23
Aggregate regression train score with FS: 0.01755192821102436, test score: -3.9100953861492224e+23
(392, 24)
Full aggregate regression train score: 0.0189971472619912, test score: -4.708287271848627e+23
Aggregate regression train score with FS: 0.0189971472619912, test score: -4.708287271848627e+23
(392, 25)
Full aggregate regression train score: 0.019167865663508366, test score: -5.310161735544999e+23
Aggregate regression train score with FS: 0.019167865663508366, test score: -5.310161735

Full aggregate regression train score: 0.01119935910065284, test score: -2.399701277674493e+23
Aggregate regression train score with FS: 0.01119935910065284, test score: -2.399701277674493e+23
(392, 13)
Full aggregate regression train score: 0.011514384987914461, test score: -2.4989472885266745e+23
Aggregate regression train score with FS: 0.011514384987914461, test score: -2.4989472885266745e+23
(392, 14)
Full aggregate regression train score: 0.01180711058431494, test score: -2.963392708614812e+23
Aggregate regression train score with FS: 0.01180711058431494, test score: -2.963392708614812e+23
(392, 15)
Full aggregate regression train score: 0.012640643691085307, test score: -3.58570129137915e+23
Aggregate regression train score with FS: 0.012640643691085307, test score: -3.58570129137915e+23
(392, 16)
Full aggregate regression train score: 0.012351825079252032, test score: -3.463540954596359e+23
Aggregate regression train score with FS: 0.012351825079252032, test score: -3.463540954

Full aggregate regression train score: 0.0013149504637928189, test score: -1.848534955306338e+22
Aggregate regression train score with FS: 0.0013149504637928189, test score: -1.848534955306338e+22
(392, 3)
Full aggregate regression train score: 0.0017609525585432495, test score: -2.059047942341157e+22
Aggregate regression train score with FS: 0.0017609525585432495, test score: -2.059047942341157e+22
(392, 4)
Full aggregate regression train score: 0.0054343751914662786, test score: -7.193095181716761e+22
Aggregate regression train score with FS: 0.0054343751914662786, test score: -7.193095181716761e+22
(392, 5)
Full aggregate regression train score: 0.00650104551560049, test score: -6.838243850596274e+22
Aggregate regression train score with FS: 0.00650104551560049, test score: -6.838243850596274e+22
(392, 6)
Full aggregate regression train score: 0.012432796964864656, test score: -1.9373336144548062e+23
Aggregate regression train score with FS: 0.012432796964864656, test score: -1.9373

Full aggregate regression train score: 0.056526779108101444, test score: -1.593617656772147e+24
Aggregate regression train score with FS: 0.056526779108101444, test score: -1.593617656772147e+24
(392, 44)
Full aggregate regression train score: 0.05535456435425323, test score: -1.49863030695235e+24
Aggregate regression train score with FS: 0.05535456435425323, test score: -1.49863030695235e+24
(392, 45)
Full aggregate regression train score: 0.057231396476273066, test score: -1.4755142641288226e+24
Aggregate regression train score with FS: 0.057231396476273066, test score: -1.4755142641288226e+24
(392, 46)
Full aggregate regression train score: 0.06220561300757821, test score: -1.740716466895173e+24
Aggregate regression train score with FS: 0.06220561300757821, test score: -1.740716466895173e+24
(392, 47)
Full aggregate regression train score: 0.06255523953303666, test score: -1.8906925809464322e+24
Aggregate regression train score with FS: 0.06255523953303666, test score: -1.8906925809

In [62]:
r = [-2.3747228845071894,-1.0695780848356164,-8.092929199055998,-3.920646329423415,-3.4967153473170287]
compute_CI(r,5)

-3.7909183690278496 +- 2.075083186707873


## LDA for class

## Supervised PCA

In [26]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import utils
from sklearn.preprocessing import KernelCenterer, scale
from sklearn.metrics.pairwise import pairwise_kernels
from scipy import linalg
from scipy.sparse.linalg import eigsh as ssl_eigsh

class spca(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_components, kernel="linear", eigen_solver='auto', 
                 max_iterations=None, gamma=0, degree=3, coef0=1, alpha=1.0, 
                 tolerance=0, fit_inverse_transform=False):
        
        self._num_components = num_components
        self._gamma = gamma
        self._tolerance = tolerance
        self._fit_inverse_transform = fit_inverse_transform
        self._max_iterations = max_iterations
        self._degree = degree
        self._kernel = kernel
        self._eigen_solver = eigen_solver
        self._coef0 = coef0
        self._centerer = KernelCenterer()
        self._alpha = alpha
        self._alphas = []
        self._lambdas = []
        
        
    def _get_kernel(self, X, Y=None):
        # Returns a kernel matrix K such that K_{i, j} is the kernel between the ith and jth vectors 
        # of the given matrix X, if Y is None. 
        
        # If Y is not None, then K_{i, j} is the kernel between the ith array from X and the jth array from Y.
        
        # valid kernels are 'linear, rbf, poly, sigmoid, precomputed'
        
        args = {"gamma": self._gamma, "degree": self._degree, "coef0": self._coef0}
        
        return pairwise_kernels(X, Y, metric=self._kernel, n_jobs=-1, filter_params=True, **args)
    
    
    
    def _fit(self, X, Y):
        
        # calculate kernel matrix of the labels Y and centre it and call it K (=H.L.H)
        K = self._centerer.fit_transform(self._get_kernel(Y))
        
        # deciding on the number of components to use
        if self._num_components is not None:
            num_components = min(K.shape[0], self._num_components)
        else:
            num_components = self.K.shape[0]
        
        # Scale X
        # scaled_X = scale(X)
        
        # calculate the eigen values and eigen vectors for X^T.K.X
        Q = (X.T).dot(K).dot(X)
        
        # If n_components is much less than the number of training samples, 
        # arpack may be more efficient than the dense eigensolver.
        if (self._eigen_solver=='auto'):
            if (Q.shape[0]/num_components) > 20:
                eigen_solver = 'arpack'
            else:
                eigen_solver = 'dense'
        else:
            eigen_solver = self._eigen_solver
        
        if eigen_solver == 'dense':
            # Return the eigenvalues (in ascending order) and eigenvectors of a Hermitian or symmetric matrix.
            self._lambdas, self._alphas = linalg.eigh(Q, eigvals=(Q.shape[0] - num_components, Q.shape[0] - 1))
            # argument eigvals = Indexes of the smallest and largest (in ascending order) eigenvalues
        
        elif eigen_solver == 'arpack':
            # deprecated :: self._lambdas, self._alphas = utils.arpack.eigsh(A=Q, num_components, which="LA", tol=self._tolerance)
            self._lambdas, self._alphas = ssl_eigsh(A=Q, k=num_components, which="LA", tol=self._tolerance)
            
        indices = self._lambdas.argsort()[::-1]
        
        self._lambdas = self._lambdas[indices]
        self._lambdas = self._lambdas[self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self._alphas = self._alphas[:, indices]
        #return self._alphas
        self._alphas = self._alphas[:, self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self.X_fit = X

        
    def _transform(self):
        return self.X_fit.dot(self._alphas)
        
        
    def transform(self, X):
        return X.dot(self._alphas)
        
        
    def fit(self, X, Y):
        self._fit(X,Y)
        return
        
        
    def fit_and_transform(self, X, Y):
        self.fit(X, Y)
        return self._transform()
    

In [41]:
## r2 = []
for kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel='linear', degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
            r2.append([curr_seed,actual_r2])
            print(test_reduced.shape)
        print(f'#########################{kernel}, {curr_seed}#############################')
    

Full aggregate regression train score: 0.1741290222837063, test score: 0.13898109011833681
Aggregate regression train score with FS: 0.1741290222837063, test score: 0.13898109011833681
(392, 1)
Full aggregate regression train score: 0.23450577274529905, test score: 0.20934459240833914
Aggregate regression train score with FS: 0.23450577274529905, test score: 0.20934459240833914
(392, 2)
Full aggregate regression train score: 0.2420096701462534, test score: 0.20353851914245258
Aggregate regression train score with FS: 0.2420096701462534, test score: 0.20353851914245258
(392, 3)
Full aggregate regression train score: 0.2573065306154485, test score: 0.21251441784914593
Aggregate regression train score with FS: 0.2573065306154485, test score: 0.21251441784914593
(392, 4)
Full aggregate regression train score: 0.5555311303150672, test score: 0.5149910922780487
Aggregate regression train score with FS: 0.5555311303150672, test score: 0.5149910922780487
(392, 5)
Full aggregate regression trai

Full aggregate regression train score: 0.871192171345743, test score: 0.7891993281271235
Aggregate regression train score with FS: 0.871192171345743, test score: 0.7891993281271235
(392, 44)
Full aggregate regression train score: 0.8905141432333799, test score: 0.7535591741081737
Aggregate regression train score with FS: 0.8905141432333799, test score: 0.7535591741081737
(392, 45)
Full aggregate regression train score: 0.8906402595684589, test score: 0.791647750304504
Aggregate regression train score with FS: 0.8906402595684589, test score: 0.791647750304504
(392, 46)
Full aggregate regression train score: 0.8982241943727391, test score: 0.7933433509520722
Aggregate regression train score with FS: 0.8982241943727391, test score: 0.7933433509520722
(392, 47)
Full aggregate regression train score: 0.8654126489370255, test score: 0.7730830526211581
Aggregate regression train score with FS: 0.8654126489370255, test score: 0.7730830526211581
(392, 48)
Full aggregate regression train score: 

Full aggregate regression train score: 0.8231505573885791, test score: 0.7196316677869086
Aggregate regression train score with FS: 0.8231505573885791, test score: 0.7196316677869086
(392, 37)
Full aggregate regression train score: 0.832209681982933, test score: 0.7276986424994512
Aggregate regression train score with FS: 0.832209681982933, test score: 0.7276986424994512
(392, 38)
Full aggregate regression train score: 0.8493900880243223, test score: 0.7685680173392065
Aggregate regression train score with FS: 0.8493900880243223, test score: 0.7685680173392065
(392, 39)
Full aggregate regression train score: 0.8365198745020455, test score: 0.7456872407694828
Aggregate regression train score with FS: 0.8365198745020455, test score: 0.7456872407694828
(392, 40)
Full aggregate regression train score: 0.8467383769368334, test score: 0.7555770422597623
Aggregate regression train score with FS: 0.8467383769368334, test score: 0.7555770422597623
(392, 41)
Full aggregate regression train score

Full aggregate regression train score: 0.8286678403056384, test score: 0.7508757660839012
Aggregate regression train score with FS: 0.8286678403056384, test score: 0.7508757660839012
(392, 30)
Full aggregate regression train score: 0.792037667416023, test score: 0.7462400214154916
Aggregate regression train score with FS: 0.792037667416023, test score: 0.7462400214154916
(392, 31)
Full aggregate regression train score: 0.8465828210546839, test score: 0.7641919856265247
Aggregate regression train score with FS: 0.8465828210546839, test score: 0.7641919856265247
(392, 32)
Full aggregate regression train score: 0.8477867146738003, test score: 0.7314372007068709
Aggregate regression train score with FS: 0.8477867146738003, test score: 0.7314372007068709
(392, 33)
Full aggregate regression train score: 0.876055269843034, test score: 0.8045870403001866
Aggregate regression train score with FS: 0.876055269843034, test score: 0.8045870403001866
(392, 34)
Full aggregate regression train score: 

Full aggregate regression train score: 0.7100830195536879, test score: 0.7011818375356011
Aggregate regression train score with FS: 0.7100830195536879, test score: 0.7011818375356011
(392, 23)
Full aggregate regression train score: 0.8346285654704757, test score: 0.7164404950758257
Aggregate regression train score with FS: 0.8346285654704757, test score: 0.7164404950758257
(392, 24)
Full aggregate regression train score: 0.7001758662840856, test score: 0.5437476524169265
Aggregate regression train score with FS: 0.7001758662840856, test score: 0.5437476524169265
(392, 25)
Full aggregate regression train score: 0.8533263940382443, test score: 0.718246681430145
Aggregate regression train score with FS: 0.8533263940382443, test score: 0.718246681430145
(392, 26)
Full aggregate regression train score: 0.8424833150072146, test score: 0.650317224903238
Aggregate regression train score with FS: 0.8424833150072146, test score: 0.650317224903238
(392, 27)
Full aggregate regression train score: 

...

Full aggregate regression train score: 0.7479631475054942, test score: 0.7140448949231462
Aggregate regression train score with FS: 0.7479631475054942, test score: 0.7140448949231462
(392, 10)
Full aggregate regression train score: 0.7561626545768851, test score: 0.664252116229047
Aggregate regression train score with FS: 0.7561626545768851, test score: 0.664252116229047
(392, 11)
Full aggregate regression train score: 0.6719158342131617, test score: 0.5210204968077916
Aggregate regression train score with FS: 0.6719158342131617, test score: 0.5210204968077916
(392, 12)
Full aggregate regression train score: 0.7752303963340491, test score: 0.699326203247304
Aggregate regression train score with FS: 0.7752303963340491, test score: 0.699326203247304
(392, 13)
Full aggregate regression train score: 0.6770322211667594, test score: 0.5399172447652301
Aggregate regression train score with FS: 0.6770322211667594, test score: 0.5399172447652301
(392, 14)
Full aggregate regression train score: 

Full aggregate regression train score: 0.4767415406501233, test score: 0.3421132832278704
Aggregate regression train score with FS: 0.4767415406501233, test score: 0.3421132832278704
(392, 3)
Full aggregate regression train score: 0.3653895191222477, test score: 0.26778385158657425
Aggregate regression train score with FS: 0.3653895191222477, test score: 0.26778385158657425
(392, 4)
Full aggregate regression train score: 0.3731216726558805, test score: 0.2537412811801041
Aggregate regression train score with FS: 0.3731216726558805, test score: 0.2537412811801041
(392, 5)
Full aggregate regression train score: 0.6076374012509498, test score: 0.5531577855985776
Aggregate regression train score with FS: 0.6076374012509498, test score: 0.5531577855985776
(392, 6)
Full aggregate regression train score: 0.5801261592769484, test score: 0.38546410862875236
Aggregate regression train score with FS: 0.5801261592769484, test score: 0.38546410862875236
(392, 7)
Full aggregate regression train scor

Full aggregate regression train score: 0.8703791042735697, test score: 0.7811047187578156
Aggregate regression train score with FS: 0.8703791042735697, test score: 0.7811047187578156
(392, 46)
Full aggregate regression train score: 0.8728023633926515, test score: 0.8058955910033357
Aggregate regression train score with FS: 0.8728023633926515, test score: 0.8058955910033357
(392, 47)
Full aggregate regression train score: 0.8618452934696434, test score: 0.821973129028372
Aggregate regression train score with FS: 0.8618452934696434, test score: 0.821973129028372
(392, 48)
Full aggregate regression train score: 0.8681845005909965, test score: 0.7639493995631492
Aggregate regression train score with FS: 0.8681845005909965, test score: 0.7639493995631492
(392, 49)
Full aggregate regression train score: 0.8551917547761881, test score: 0.6144637546604128
Aggregate regression train score with FS: 0.8551917547761881, test score: 0.6144637546604128
(392, 50)
#########################sigmoid, 2##

Full aggregate regression train score: 0.8894437437452026, test score: 0.8377079296490393
Aggregate regression train score with FS: 0.8894437437452026, test score: 0.8377079296490393
(392, 39)
Full aggregate regression train score: 0.8897184965860294, test score: 0.8045752551066067
Aggregate regression train score with FS: 0.8897184965860294, test score: 0.8045752551066067
(392, 40)
Full aggregate regression train score: 0.878770499136978, test score: 0.8244507506085962
Aggregate regression train score with FS: 0.878770499136978, test score: 0.8244507506085962
(392, 41)
Full aggregate regression train score: 0.8850035597339106, test score: 0.8328399318739931
Aggregate regression train score with FS: 0.8850035597339106, test score: 0.8328399318739931
(392, 42)
Full aggregate regression train score: 0.8748929066400393, test score: 0.8163261861298818
Aggregate regression train score with FS: 0.8748929066400393, test score: 0.8163261861298818
(392, 43)
Full aggregate regression train score

Full aggregate regression train score: 0.8412574270122315, test score: 0.7779705016796206
Aggregate regression train score with FS: 0.8412574270122315, test score: 0.7779705016796206
(392, 32)
Full aggregate regression train score: 0.84006092889611, test score: 0.7062083167783682
Aggregate regression train score with FS: 0.84006092889611, test score: 0.7062083167783682
(392, 33)
Full aggregate regression train score: 0.839450449824836, test score: 0.7503917580852606
Aggregate regression train score with FS: 0.839450449824836, test score: 0.7503917580852606
(392, 34)
Full aggregate regression train score: 0.859656542685142, test score: 0.7769375802554996
Aggregate regression train score with FS: 0.859656542685142, test score: 0.7769375802554996
(392, 35)
Full aggregate regression train score: 0.8501278037871882, test score: 0.7801390777068545
Aggregate regression train score with FS: 0.8501278037871882, test score: 0.7801390777068545
(392, 36)
Full aggregate regression train score: 0.82

In [66]:
linear_r2 = [0.868712798267652,0.8505925691557306,0.8441932595996989,0.8145828853292557,0.8464935517171148]
poly_r2 = [0.8458989757623416,0.8530458122995871,0.8418390773843332,0.8490346137405589,0.83704655597334]
sigmoid_r2 = [0.8376847401038159,0.8319222128595791,0.8453618172161299,0.8451255695548574,0.8418665935034199]

linear_n = [34,49,38,41,37]
poly_n = [38,40,41,46,44]
sigmoid_n = [35,46,27,47,46]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)

0.8449150128138904 +- 0.015291115122897407
0.845373007032032 +- 0.00487050169857498
0.8403921866475604 +- 0.004440976634235046
39.8 +- 4.483218843643482
41.8 +- 2.5038952054748616
40.2 +- 6.955100804445611


## Kernel PCA

In [42]:
from sklearn.decomposition import KernelPCA

In [65]:
## r2 = []

for curr_kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        best_r2 = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
            r2.append([curr_seed,actual_r2])
            print(test_reduced.shape)
            if actual_r2> best_r2:
                best_r2=actual_r2
                best_num = test_reduced.shape
        print(best_r2)
        print(best_num)
        print(f'#########################{kernel}, {curr_seed}#############################')


Full aggregate regression train score: 0.16079042739121874, test score: 0.12993644314515795
Aggregate regression train score with FS: 0.16079042739121874, test score: 0.12993644314515795
(392, 1)
Full aggregate regression train score: 0.16085423929353138, test score: 0.13036382070524388
Aggregate regression train score with FS: 0.16085423929353138, test score: 0.13036382070524388
(392, 2)
Full aggregate regression train score: 0.16559479967107849, test score: 0.14326083228577968
Aggregate regression train score with FS: 0.16559479967107849, test score: 0.14326083228577968
(392, 3)
Full aggregate regression train score: 0.16924479045653051, test score: 0.12149552573197653
Aggregate regression train score with FS: 0.16924479045653051, test score: 0.12149552573197653
(392, 4)
Full aggregate regression train score: 0.17201121730306745, test score: 0.11984167511421617
Aggregate regression train score with FS: 0.17201121730306745, test score: 0.11984167511421617
(392, 5)
Full aggregate regre

Full aggregate regression train score: 0.8918838835211734, test score: 0.7966540023280457
Aggregate regression train score with FS: 0.8918838835211734, test score: 0.7966540023280457
(392, 46)
Full aggregate regression train score: 0.8919640317479286, test score: 0.7964687763939587
Aggregate regression train score with FS: 0.8919640317479286, test score: 0.7964687763939587
(392, 47)
Full aggregate regression train score: 0.8922401958047046, test score: 0.7923988872878782
Aggregate regression train score with FS: 0.8922401958047046, test score: 0.7923988872878782
(392, 48)
Full aggregate regression train score: 0.8930273774656167, test score: 0.7975394946743344
Aggregate regression train score with FS: 0.8930273774656167, test score: 0.7975394946743344
(392, 49)
Full aggregate regression train score: 0.8947401743125525, test score: 0.8056856403511645
Aggregate regression train score with FS: 0.8947401743125525, test score: 0.8056856403511645
(392, 50)
0.8056856403511645
(392, 50)
######

Full aggregate regression train score: 0.8706069710244367, test score: 0.7873494584240127
Aggregate regression train score with FS: 0.8706069710244367, test score: 0.7873494584240127
(392, 40)
Full aggregate regression train score: 0.8718247127526464, test score: 0.7930057944325594
Aggregate regression train score with FS: 0.8718247127526464, test score: 0.7930057944325594
(392, 41)
Full aggregate regression train score: 0.8722585290569378, test score: 0.7912513907604777
Aggregate regression train score with FS: 0.8722585290569378, test score: 0.7912513907604777
(392, 42)
Full aggregate regression train score: 0.8778854771251361, test score: 0.7833869723302181
Aggregate regression train score with FS: 0.8778854771251361, test score: 0.7833869723302181
(392, 43)
Full aggregate regression train score: 0.8802238003482732, test score: 0.7852831165119778
Aggregate regression train score with FS: 0.8802238003482732, test score: 0.7852831165119778
(392, 44)
Full aggregate regression train sco

Full aggregate regression train score: 0.8695885360369047, test score: 0.7969188157358895
Aggregate regression train score with FS: 0.8695885360369047, test score: 0.7969188157358895
(392, 32)
Full aggregate regression train score: 0.8721166600883571, test score: 0.7942288346378692
Aggregate regression train score with FS: 0.8721166600883571, test score: 0.7942288346378692
(392, 33)
Full aggregate regression train score: 0.8722404262597467, test score: 0.7933707958550724
Aggregate regression train score with FS: 0.8722404262597467, test score: 0.7933707958550724
(392, 34)
Full aggregate regression train score: 0.8737712349899022, test score: 0.7928367866142689
Aggregate regression train score with FS: 0.8737712349899022, test score: 0.7928367866142689
(392, 35)
Full aggregate regression train score: 0.8855635350451261, test score: 0.7891211595141636
Aggregate regression train score with FS: 0.8855635350451261, test score: 0.7891211595141636
(392, 36)
Full aggregate regression train sco

Full aggregate regression train score: 0.8671891180608311, test score: 0.77734549206219
Aggregate regression train score with FS: 0.8671891180608311, test score: 0.77734549206219
(392, 26)
Full aggregate regression train score: 0.8684736939654564, test score: 0.7826840859535862
Aggregate regression train score with FS: 0.8684736939654564, test score: 0.7826840859535862
(392, 27)
Full aggregate regression train score: 0.8691714078352741, test score: 0.7832227892168654
Aggregate regression train score with FS: 0.8691714078352741, test score: 0.7832227892168654
(392, 28)
Full aggregate regression train score: 0.8747299708852777, test score: 0.7886362300965946
Aggregate regression train score with FS: 0.8747299708852777, test score: 0.7886362300965946
(392, 29)
Full aggregate regression train score: 0.8749287505918444, test score: 0.787675262795545
Aggregate regression train score with FS: 0.8749287505918444, test score: 0.787675262795545
(392, 30)
Full aggregate regression train score: 0.

...

Full aggregate regression train score: 0.2722068873689639, test score: -0.08582154052539637
Aggregate regression train score with FS: 0.2722068873689639, test score: -0.08582154052539637
(392, 20)
Full aggregate regression train score: 0.3040132370767039, test score: -0.061068804320697945
Aggregate regression train score with FS: 0.3040132370767039, test score: -0.061068804320697945
(392, 21)
Full aggregate regression train score: 0.3178665724993497, test score: -0.05020413897070197
Aggregate regression train score with FS: 0.3178665724993497, test score: -0.05020413897070197
(392, 22)
Full aggregate regression train score: 0.34903653763573816, test score: 0.013280248854981136
Aggregate regression train score with FS: 0.34903653763573816, test score: 0.013280248854981136
(392, 23)
Full aggregate regression train score: 0.3661058206208627, test score: 0.04515503999329895
Aggregate regression train score with FS: 0.3661058206208627, test score: 0.04515503999329895
(392, 24)
Full aggregat

Full aggregate regression train score: 0.18791802461252838, test score: 0.09018129494781524
Aggregate regression train score with FS: 0.18791802461252838, test score: 0.09018129494781524
(392, 12)
Full aggregate regression train score: 0.18830129394574557, test score: 0.08251644216459009
Aggregate regression train score with FS: 0.18830129394574557, test score: 0.08251644216459009
(392, 13)
Full aggregate regression train score: 0.1906175144308474, test score: 0.08065514099466276
Aggregate regression train score with FS: 0.1906175144308474, test score: 0.08065514099466276
(392, 14)
Full aggregate regression train score: 0.19061755435728633, test score: 0.08062096287985587
Aggregate regression train score with FS: 0.19061755435728633, test score: 0.08062096287985587
(392, 15)
Full aggregate regression train score: 0.19642851992169097, test score: 0.07405404480757383
Aggregate regression train score with FS: 0.19642851992169097, test score: 0.07405404480757383
(392, 16)
Full aggregate re

Full aggregate regression train score: 0.2067261458917634, test score: 0.04723097433032375
Aggregate regression train score with FS: 0.2067261458917634, test score: 0.04723097433032375
(392, 6)
Full aggregate regression train score: 0.20859716985778654, test score: 0.031003700828052705
Aggregate regression train score with FS: 0.20859716985778654, test score: 0.031003700828052705
(392, 7)
Full aggregate regression train score: 0.21870630931044954, test score: 0.020934614940129825
Aggregate regression train score with FS: 0.21870630931044954, test score: 0.020934614940129825
(392, 8)
Full aggregate regression train score: 0.21871964255528642, test score: 0.020851268584460625
Aggregate regression train score with FS: 0.21871964255528642, test score: 0.020851268584460625
(392, 9)
Full aggregate regression train score: 0.22508050584226302, test score: -0.01535068504279935
Aggregate regression train score with FS: 0.22508050584226302, test score: -0.01535068504279935
(392, 10)
Full aggregat

Full aggregate regression train score: 0.8272389026495367, test score: 0.3484153301240741
Aggregate regression train score with FS: 0.8272389026495367, test score: 0.3484153301240741
(392, 50)
0.48256387071500084
(392, 35)
#########################sigmoid, 3#############################
Full aggregate regression train score: 0.1250991156453135, test score: 0.11390543672806663
Aggregate regression train score with FS: 0.1250991156453135, test score: 0.11390543672806663
(392, 1)
Full aggregate regression train score: 0.12566020593672922, test score: 0.11465757958112932
Aggregate regression train score with FS: 0.12566020593672922, test score: 0.11465757958112932
(392, 2)
Full aggregate regression train score: 0.1284802511876001, test score: 0.09874758476164591
Aggregate regression train score with FS: 0.1284802511876001, test score: 0.09874758476164591
(392, 3)
Full aggregate regression train score: 0.14517092434694034, test score: 0.0521672732246784
Aggregate regression train score with

Full aggregate regression train score: 0.8286038973767591, test score: 0.6369612972710155
Aggregate regression train score with FS: 0.8286038973767591, test score: 0.6369612972710155
(392, 42)
Full aggregate regression train score: 0.8301185001170479, test score: 0.6428108199321012
Aggregate regression train score with FS: 0.8301185001170479, test score: 0.6428108199321012
(392, 43)
Full aggregate regression train score: 0.8302106272879264, test score: 0.6429177926761115
Aggregate regression train score with FS: 0.8302106272879264, test score: 0.6429177926761115
(392, 44)
Full aggregate regression train score: 0.8302205712808863, test score: 0.6422226648023881
Aggregate regression train score with FS: 0.8302205712808863, test score: 0.6422226648023881
(392, 45)
Full aggregate regression train score: 0.8306371217751162, test score: 0.6437706362747826
Aggregate regression train score with FS: 0.8306371217751162, test score: 0.6437706362747826
(392, 46)
Full aggregate regression train sco

In [68]:
linear_r2 = [0.8056856403511645,0.796884618064896,0.8005418847526391,0.8053003786058056,0.7866743278769734]
poly_r2 = [0.12871718886410533,0.13096703064806037,0.11477440787056137,0.1105553015142805,0.12071432938945648]
rbf_r2 = [0.5129440299781838,0.4921994153114828,0.38277721397441544,0.5111699272176614,0.44123136879809066]
sigmoid_r2 = [0.5619288583583297,0.5711379570032745,0.5297745454938106,0.48256387071500084,0.6437706362747826]

linear_n = [50,28,27,42,31]
poly_n = [5,14,7,7,49]
rbf_n = [50,50,50,50,49]
sigmoid_n = [50,47,31,35,46]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)
compute_CI(rbf_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)
compute_CI(rbf_n,5)


0.7990173699302957 +- 0.006112755686939906
0.12114565165729281 +- 0.006865283955022416
0.5578351735690397 +- 0.04643470235801806
0.46806439105596687 +- 0.043736875924430374
35.6 +- 7.851751193205245
16.4 +- 14.536806141652985
41.8 +- 6.498220310208018
49.8 +- 0.35061545887196705


## Isomap

In [69]:
from sklearn.manifold import Isomap
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
        if actual_r2> best_r2:
            best_r2=actual_r2
            best_num = test_reduced.shape
    print(best_r2)
    print(best_num)
        

Full aggregate regression train score: 0.15930050943465335, test score: 0.13307443013569742
Aggregate regression train score with FS: 0.15930050943465335, test score: 0.13307443013569742
(392, 1)
Full aggregate regression train score: 0.15982584470828776, test score: 0.1358727219795165
Aggregate regression train score with FS: 0.15982584470828776, test score: 0.1358727219795165
(392, 2)
Full aggregate regression train score: 0.16931396510315877, test score: 0.1543317261926589
Aggregate regression train score with FS: 0.16931396510315877, test score: 0.1543317261926589
(392, 3)
Full aggregate regression train score: 0.17177946820838774, test score: 0.1498748643151383
Aggregate regression train score with FS: 0.17177946820838774, test score: 0.1498748643151383
(392, 4)
Full aggregate regression train score: 0.18941347308363288, test score: 0.13658230949806893
Aggregate regression train score with FS: 0.18941347308363288, test score: 0.13658230949806893
(392, 5)
Full aggregate regression 

Aggregate regression train score with FS: 0.37394568557817587, test score: -0.0583963268629859
(392, 43)
Full aggregate regression train score: 0.3779984505427538, test score: -0.05163380603604306
Aggregate regression train score with FS: 0.3779984505427538, test score: -0.05163380603604306
(392, 44)
Full aggregate regression train score: 0.3784684516963518, test score: -0.05066422385618119
Aggregate regression train score with FS: 0.3784684516963518, test score: -0.05066422385618119
(392, 45)
Full aggregate regression train score: 0.3789253688578894, test score: -0.0488172151530859
Aggregate regression train score with FS: 0.3789253688578894, test score: -0.0488172151530859
(392, 46)
Full aggregate regression train score: 0.3789531048192786, test score: -0.049999095871924526
Aggregate regression train score with FS: 0.3789531048192786, test score: -0.049999095871924526
(392, 47)
Full aggregate regression train score: 0.38449528277513434, test score: -0.06523266841184805
Aggregate regr

Full aggregate regression train score: 0.2740543980902621, test score: 0.06281400662198233
Aggregate regression train score with FS: 0.2740543980902621, test score: 0.06281400662198233
(392, 37)
Full aggregate regression train score: 0.28515737583809575, test score: 0.03992548157155773
Aggregate regression train score with FS: 0.28515737583809575, test score: 0.03992548157155773
(392, 38)
Full aggregate regression train score: 0.28544166102846613, test score: 0.039836174807581726
Aggregate regression train score with FS: 0.28544166102846613, test score: 0.039836174807581726
(392, 39)
Full aggregate regression train score: 0.28760851674806165, test score: 0.035910174652415416
Aggregate regression train score with FS: 0.28760851674806165, test score: 0.035910174652415416
(392, 40)
Full aggregate regression train score: 0.3051373065963785, test score: 0.05559735172271507
Aggregate regression train score with FS: 0.3051373065963785, test score: 0.05559735172271507
(392, 41)
Full aggregate 

Full aggregate regression train score: 0.3206516299434977, test score: -0.0023255917211673083
Aggregate regression train score with FS: 0.3206516299434977, test score: -0.0023255917211673083
(392, 29)
Full aggregate regression train score: 0.32597972760861804, test score: -0.00021188390882498354
Aggregate regression train score with FS: 0.32597972760861804, test score: -0.00021188390882498354
(392, 30)
Full aggregate regression train score: 0.32766953963014644, test score: -0.010139869700045434
Aggregate regression train score with FS: 0.32766953963014644, test score: -0.010139869700045434
(392, 31)
Full aggregate regression train score: 0.3304770280765521, test score: 0.0033301480111564974
Aggregate regression train score with FS: 0.3304770280765521, test score: 0.0033301480111564974
(392, 32)
Full aggregate regression train score: 0.3307057190892698, test score: 0.0032104724804333795
Aggregate regression train score with FS: 0.3307057190892698, test score: 0.0032104724804333795
(392,

Full aggregate regression train score: 0.3219545314713764, test score: 0.007353375833529574
Aggregate regression train score with FS: 0.3219545314713764, test score: 0.007353375833529574
(392, 21)
Full aggregate regression train score: 0.3294041342665647, test score: 0.01753974752809473
Aggregate regression train score with FS: 0.3294041342665647, test score: 0.01753974752809473
(392, 22)
Full aggregate regression train score: 0.3304778950662959, test score: 0.016298324972666478
Aggregate regression train score with FS: 0.3304778950662959, test score: 0.016298324972666478
(392, 23)
Full aggregate regression train score: 0.3319918301067719, test score: 0.00932661331312068
Aggregate regression train score with FS: 0.3319918301067719, test score: 0.00932661331312068
(392, 24)
Full aggregate regression train score: 0.3466695467129064, test score: 0.014202886069535792
Aggregate regression train score with FS: 0.3466695467129064, test score: 0.014202886069535792
(392, 25)
Full aggregate regr

Full aggregate regression train score: 0.20897796218456344, test score: 0.05435520585916975
Aggregate regression train score with FS: 0.20897796218456344, test score: 0.05435520585916975
(392, 13)
Full aggregate regression train score: 0.21965785148923722, test score: 0.062192780290372784
Aggregate regression train score with FS: 0.21965785148923722, test score: 0.062192780290372784
(392, 14)
Full aggregate regression train score: 0.21992552016352196, test score: 0.06279672027956107
Aggregate regression train score with FS: 0.21992552016352196, test score: 0.06279672027956107
(392, 15)
Full aggregate regression train score: 0.22510285108784722, test score: 0.05083521044911887
Aggregate regression train score with FS: 0.22510285108784722, test score: 0.05083521044911887
(392, 16)
Full aggregate regression train score: 0.2253421839534422, test score: 0.04814474367340593
Aggregate regression train score with FS: 0.2253421839534422, test score: 0.04814474367340593
(392, 17)
Full aggregate 

In [70]:
best_isomap_r2 = [0.1543317261926589,0.12317495529328326,0.14547688540586667,0.13566723304732098,0.11829610779219302]

best_isomap_n = [3,3,3,1,3]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.13538938154626456 +- 0.011773187147230284
2.6 +- 0.7012309177439341


## LLE

In [71]:
from sklearn.manifold import LocallyLinearEmbedding as LLE
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
        if actual_r2> best_r2:
            best_r2=actual_r2
            best_num = test_reduced.shape
    print(best_r2)
    print(best_num)

Full aggregate regression train score: 0.16383324399282928, test score: 0.13356575846613894
Aggregate regression train score with FS: 0.16383324399282928, test score: 0.13356575846613894
(392, 1)
Full aggregate regression train score: 0.16914201076343904, test score: 0.14321896252201682
Aggregate regression train score with FS: 0.16914201076343904, test score: 0.14321896252201682
(392, 2)
Full aggregate regression train score: 0.16956359787877684, test score: 0.1427525721895775
Aggregate regression train score with FS: 0.16956359787877684, test score: 0.1427525721895775
(392, 3)
Full aggregate regression train score: 0.17071554140294998, test score: 0.14126406151106308
Aggregate regression train score with FS: 0.17071554140294998, test score: 0.14126406151106308
(392, 4)
Full aggregate regression train score: 0.19947937165731044, test score: 0.11845527867285599
Aggregate regression train score with FS: 0.19947937165731044, test score: 0.11845527867285599
(392, 5)
Full aggregate regress

Full aggregate regression train score: 0.39245031010456, test score: 0.06821159971610369
Aggregate regression train score with FS: 0.39245031010456, test score: 0.06821159971610369
(392, 43)
Full aggregate regression train score: 0.39350994217256474, test score: 0.06983931619250028
Aggregate regression train score with FS: 0.39350994217256474, test score: 0.06983931619250028
(392, 44)
Full aggregate regression train score: 0.39445685463149205, test score: 0.06803759446964353
Aggregate regression train score with FS: 0.39445685463149205, test score: 0.06803759446964353
(392, 45)
Full aggregate regression train score: 0.41132081613240923, test score: 0.05589645249291597
Aggregate regression train score with FS: 0.41132081613240923, test score: 0.05589645249291597
(392, 46)
Full aggregate regression train score: 0.41509856268549894, test score: 0.053033826654585114
Aggregate regression train score with FS: 0.41509856268549894, test score: 0.053033826654585114
(392, 47)
Full aggregate regr

Full aggregate regression train score: 0.3125085024356776, test score: 0.07264661368692293
Aggregate regression train score with FS: 0.3125085024356776, test score: 0.07264661368692293
(392, 35)
Full aggregate regression train score: 0.33470927695258634, test score: 0.08374515184335096
Aggregate regression train score with FS: 0.33470927695258634, test score: 0.08374515184335096
(392, 36)
Full aggregate regression train score: 0.3353109419680862, test score: 0.08273500318318938
Aggregate regression train score with FS: 0.3353109419680862, test score: 0.08273500318318938
(392, 37)
Full aggregate regression train score: 0.3730404119644126, test score: 0.08731357916068672
Aggregate regression train score with FS: 0.3730404119644126, test score: 0.08731357916068672
(392, 38)
Full aggregate regression train score: 0.3730449755511126, test score: 0.08728055663530265
Aggregate regression train score with FS: 0.3730449755511126, test score: 0.08728055663530265
(392, 39)
Full aggregate regressi

Full aggregate regression train score: 0.31215181324244945, test score: 0.09546181418961042
Aggregate regression train score with FS: 0.31215181324244945, test score: 0.09546181418961042
(392, 27)
Full aggregate regression train score: 0.31221314180284354, test score: 0.09541812107959013
Aggregate regression train score with FS: 0.31221314180284354, test score: 0.09541812107959013
(392, 28)
Full aggregate regression train score: 0.31221753173801536, test score: 0.0953548198180173
Aggregate regression train score with FS: 0.31221753173801536, test score: 0.0953548198180173
(392, 29)
Full aggregate regression train score: 0.31228223593710236, test score: 0.0949941867311982
Aggregate regression train score with FS: 0.31228223593710236, test score: 0.0949941867311982
(392, 30)
Full aggregate regression train score: 0.3144767850362882, test score: 0.10065861769123585
Aggregate regression train score with FS: 0.3144767850362882, test score: 0.10065861769123585
(392, 31)
Full aggregate regres

Full aggregate regression train score: 0.2975796945647662, test score: 0.07171769052187549
Aggregate regression train score with FS: 0.2975796945647662, test score: 0.07171769052187549
(392, 19)
Full aggregate regression train score: 0.30340909183444786, test score: 0.06361033182436182
Aggregate regression train score with FS: 0.30340909183444786, test score: 0.06361033182436182
(392, 20)
Full aggregate regression train score: 0.30528072201951884, test score: 0.06569253460393232
Aggregate regression train score with FS: 0.30528072201951884, test score: 0.06569253460393232
(392, 21)
Full aggregate regression train score: 0.3052913065779185, test score: 0.06591281644125513
Aggregate regression train score with FS: 0.3052913065779185, test score: 0.06591281644125513
(392, 22)
Full aggregate regression train score: 0.3189828519743676, test score: 0.07396726643602225
Aggregate regression train score with FS: 0.3189828519743676, test score: 0.07396726643602225
(392, 23)
Full aggregate regres

Full aggregate regression train score: 0.17026651958941452, test score: 0.03430066403726706
Aggregate regression train score with FS: 0.17026651958941452, test score: 0.03430066403726706
(392, 11)
Full aggregate regression train score: 0.1706430025466874, test score: 0.030340441522470396
Aggregate regression train score with FS: 0.1706430025466874, test score: 0.030340441522470396
(392, 12)
Full aggregate regression train score: 0.17781476907438143, test score: 0.04407691219370724
Aggregate regression train score with FS: 0.17781476907438143, test score: 0.04407691219370724
(392, 13)
Full aggregate regression train score: 0.18212109559157386, test score: 0.046578768200216225
Aggregate regression train score with FS: 0.18212109559157386, test score: 0.046578768200216225
(392, 14)
Full aggregate regression train score: 0.18244369541397165, test score: 0.047016653616709925
Aggregate regression train score with FS: 0.18244369541397165, test score: 0.047016653616709925
(392, 15)
Full aggreg

In [72]:
best_lle_r2 = [0.14321896252201682,0.11426850979031511,0.11743792174159084,0.10009713132646747,0.09956426468089519]
best_lle_n = [2,45,15,13,2]

compute_CI(best_lle_r2,5)
compute_CI(best_lle_n,5)


0.11491735801225708 +- 0.013931790613353772
15.4 +- 13.810422035549818


## UMAP

In [7]:
# explain 95% of variance
import umap

for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = umap.UMAP(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
        if actual_r2> best_r2:
            best_r2=actual_r2
            best_num = test_reduced.shape
    print(best_r2)
    print(best_num)
    

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Full aggregate regression train score: 0.10086696718347887, test score: 0.10124765156332449
Aggregate regression train score with FS: 0.10086696718347887, test score: 0.10124765156332449
(392, 1)
Full aggregate regression train score: 0.13680905842208468, test score: 0.12719181574004512
Aggregate regression train score with FS: 0.13680905842208468, test score: 0.12719181574004512
(392, 2)
Full aggregate regression train score: 0.14253675972713398, test score: 0.11124572537663391
Aggregate regression train score with FS: 0.14253675972713398, test score: 0.11124572537663391
(392, 3)
Full aggregate regression train score: 0.1309676357340328, test score: 0.1317384847633003
Aggregate regression train score with FS: 0.1309676357340328, test score: 0.1317384847633003
(392, 4)
Full aggregate regression train score: 0.140398856236303, test score: 0.13449664652064008
Aggregate regression train score with FS: 0.140398856236303, test score: 0.13449664652064008
(392, 5)
Full aggregate regression tr

Full aggregate regression train score: 0.32351651677837545, test score: 0.044566428309628425
Aggregate regression train score with FS: 0.32351651677837545, test score: 0.044566428309628425
(392, 43)
Full aggregate regression train score: 0.3500482331787822, test score: -0.08493919185832799
Aggregate regression train score with FS: 0.3500482331787822, test score: -0.08493919185832799
(392, 44)
Full aggregate regression train score: 0.35794341224905124, test score: -0.0085266162870965
Aggregate regression train score with FS: 0.35794341224905124, test score: -0.0085266162870965
(392, 45)
Full aggregate regression train score: 0.3955307994471554, test score: -0.17250236753088322
Aggregate regression train score with FS: 0.3955307994471554, test score: -0.17250236753088322
(392, 46)
Full aggregate regression train score: 0.3850323964971063, test score: -0.07399662464020662
Aggregate regression train score with FS: 0.3850323964971063, test score: -0.07399662464020662
(392, 47)
Full aggregat

Full aggregate regression train score: 0.23621130761971154, test score: 0.06819961352866322
Aggregate regression train score with FS: 0.23621130761971154, test score: 0.06819961352866322
(392, 35)
Full aggregate regression train score: 0.2710137011334004, test score: -0.0020606759291446064
Aggregate regression train score with FS: 0.2710137011334004, test score: -0.0020606759291446064
(392, 36)
Full aggregate regression train score: 0.28026325906111915, test score: -0.047245738825198336
Aggregate regression train score with FS: 0.28026325906111915, test score: -0.047245738825198336
(392, 37)
Full aggregate regression train score: 0.24592141173476778, test score: 0.0039036165112037446
Aggregate regression train score with FS: 0.24592141173476778, test score: 0.0039036165112037446
(392, 38)
Full aggregate regression train score: 0.2519684519647388, test score: -0.013809223693437067
Aggregate regression train score with FS: 0.2519684519647388, test score: -0.013809223693437067
(392, 39)
F

Full aggregate regression train score: 0.2818741967203764, test score: 0.04449082918665859
Aggregate regression train score with FS: 0.2818741967203764, test score: 0.04449082918665859
(392, 27)
Full aggregate regression train score: 0.23835087664807986, test score: 0.11503133349123762
Aggregate regression train score with FS: 0.23835087664807986, test score: 0.11503133349123762
(392, 28)
Full aggregate regression train score: 0.26213576259025195, test score: 0.05425170060901352
Aggregate regression train score with FS: 0.26213576259025195, test score: 0.05425170060901352
(392, 29)
Full aggregate regression train score: 0.25103631775288093, test score: 0.021813651604693396
Aggregate regression train score with FS: 0.25103631775288093, test score: 0.021813651604693396
(392, 30)
Full aggregate regression train score: 0.27699573670513733, test score: 0.08770979549647118
Aggregate regression train score with FS: 0.27699573670513733, test score: 0.08770979549647118
(392, 31)
Full aggregate 

Full aggregate regression train score: 0.22729526298644465, test score: 0.08481661454750933
Aggregate regression train score with FS: 0.22729526298644465, test score: 0.08481661454750933
(392, 19)
Full aggregate regression train score: 0.26865800812811536, test score: 0.005621069497643849
Aggregate regression train score with FS: 0.26865800812811536, test score: 0.005621069497643849
(392, 20)
Full aggregate regression train score: 0.2577091925564071, test score: 0.08049086083763002
Aggregate regression train score with FS: 0.2577091925564071, test score: 0.08049086083763002
(392, 21)
Full aggregate regression train score: 0.27184377670050575, test score: 0.07631142849689321
Aggregate regression train score with FS: 0.27184377670050575, test score: 0.07631142849689321
(392, 22)
Full aggregate regression train score: 0.23847891317183045, test score: 0.060910593185234285
Aggregate regression train score with FS: 0.23847891317183045, test score: 0.060910593185234285
(392, 23)
Full aggregat

Full aggregate regression train score: 0.12003345450499914, test score: 0.09284908371752287
Aggregate regression train score with FS: 0.12003345450499914, test score: 0.09284908371752287
(392, 11)
Full aggregate regression train score: 0.12945213055315896, test score: 0.053276939459541706
Aggregate regression train score with FS: 0.12945213055315896, test score: 0.053276939459541706
(392, 12)
Full aggregate regression train score: 0.14921674933631268, test score: 0.05627614274514037
Aggregate regression train score with FS: 0.14921674933631268, test score: 0.05627614274514037
(392, 13)
Full aggregate regression train score: 0.11568264971025544, test score: 0.11227007466219796
Aggregate regression train score with FS: 0.11568264971025544, test score: 0.11227007466219796
(392, 14)
Full aggregate regression train score: 0.14689365669732868, test score: 0.06961928559919783
Aggregate regression train score with FS: 0.14689365669732868, test score: 0.06961928559919783
(392, 15)
Full aggregat

In [60]:
n_pca95 = [18,19,18,19,17]
r2_pca95 = [0.7850091641907997,0.7633714791597755,0.7609848606090145,0.722959063643331,0.7358441197745467]
compute_CI(n_pca95,5)
compute_CI(r2_pca95,5)

18.2 +- 0.6559414608027151
0.7536337374754934 +- 0.019166700154449642


# Classification

In [78]:
df = pd.read_csv('droughts_extended.csv')
df['mean_std'] = df.apply(lambda x: np.sign(x.mean_std),axis=1)
df_trainVal_withTar = df.iloc[:-392,:]
df_test_withTar = df.iloc[-392:,:]
df_trainVal = df.iloc[:-392,:-1]
df_test = df.iloc[-392:,:-1]
target_df_trainVal = df.iloc[:-392,-1]
target_df_test = df.iloc[-392:,-1]

## PCA

In [83]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

r2 = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    print(test_pca.shape)
    mod = LogisticRegression(random_state=0).fit(trainVal_pca, curr_df_trainVal_withTar.mean_std)
    print(mod.score(test_pca, df_test_withTar.mean_std))
    

(392, 18)
0.8469387755102041
(392, 19)
0.8571428571428571
(392, 18)
0.8545918367346939
(392, 19)
0.8418367346938775
(392, 17)
0.8571428571428571


In [84]:
n_pca95 = [18,19,18,19,17]
r2_pca95 = [0.8469387755102041,0.8571428571428571,0.8545918367346939,0.8418367346938775,0.8571428571428571]
compute_CI(n_pca95,5)
compute_CI(r2_pca95,5)

18.2 +- 0.6559414608027151
0.851530612244898 +- 0.005366563145999473


## LDA

In [87]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [93]:
from sklearn.manifold import LocallyLinearEmbedding as LLE
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    for i in range(1):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LDA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal,curr_df_trainVal_withTar.mean_std))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        print(mod.score(test_reduced, df_test_withTar.mean_std))
        

(392, 1)
0.7551020408163265
(392, 1)
0.7168367346938775
(392, 1)
0.7448979591836735
(392, 1)
0.7576530612244898
(392, 1)
0.7040816326530612


In [94]:
compute_CI([0.7551020408163265,0.7168367346938775,0.7448979591836735,0.7576530612244898,0.7040816326530612],5)

0.7357142857142858 +- 0.018793615937333617


## Kernel PCA

In [97]:
for curr_kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        best_score = 0
        best_num = 0
        print(f'#########################{curr_kernel}, {curr_seed}#############################')
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)


#########################linear, 0#############################
(392, 1)
0.625
(392, 2)
0.625
(392, 3)
0.6275510204081632
(392, 4)
0.6020408163265306
(392, 5)
0.6020408163265306
(392, 6)
0.6224489795918368
(392, 7)
0.625
(392, 8)
0.625
(392, 9)
0.6071428571428571
(392, 10)
0.6224489795918368
(392, 11)
0.6326530612244898
(392, 12)
0.6556122448979592
(392, 13)
0.6607142857142857
(392, 14)
0.7806122448979592
(392, 15)
0.7806122448979592
(392, 16)
0.8010204081632653
(392, 17)
0.8520408163265306
(392, 18)
0.8469387755102041
(392, 19)
0.8520408163265306
(392, 20)
0.8545918367346939
(392, 21)
0.8545918367346939
(392, 22)
0.8520408163265306
(392, 23)
0.8596938775510204


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 24)
0.8622448979591837
(392, 25)
0.8596938775510204
(392, 26)
0.8596938775510204
(392, 27)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8596938775510204
(392, 28)
0.8622448979591837
(392, 29)
0.8622448979591837
(392, 30)
0.8596938775510204


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.8367346938775511
(392, 50)
0.8341836734693877
0.8647959183673469
(392, 20)
#########################poly, 0#############################


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 1)
0.5331632653061225
(392, 2)
0.6224489795918368
(392, 3)
0.6326530612244898
(392, 4)
0.6301020408163265
(392, 5)
0.6326530612244898
(392, 6)
0.6326530612244898
(392, 7)
0.6173469387755102
(392, 8)
0.6173469387755102
(392, 9)
0.625
(392, 10)
0.6352040816326531
(392, 11)
0.6326530612244898
(392, 12)
0.6352040816326531
(392, 13)
0.6224489795918368
(392, 14)
0.6096938775510204
(392, 15)
0.6096938775510204
(392, 16)
0.5969387755102041
(392, 17)
0.5943877551020408
(392, 18)
0.5892857142857143
(392, 19)
0.5994897959183674
(392, 20)
0.5994897959183674
(392, 21)
0.6071428571428571
(392, 22)
0.6198979591836735
(392, 23)
0.625
(392, 24)
0.6352040816326531
(392, 25)
0.6377551020408163
(392, 26)
0.6352040816326531
(392, 27)
0.6352040816326531
(392, 28)
0.6301020408163265
(392, 29)
0.6301020408163265
(392, 30)
0.6275510204081632
(392, 31)
0.6352040816326531
(392, 32)
0.6377551020408163
(392, 33)
0.6428571428571429
(392, 34)
0.6377551020408163
(392, 35)
0.6607142857142857
(392, 36)
0.66071428

(392, 25)
0.6811224489795918
(392, 26)
0.6836734693877551
(392, 27)
0.6862244897959183
(392, 28)
0.6836734693877551
(392, 29)
0.6862244897959183
(392, 30)
0.6836734693877551
(392, 31)
0.6862244897959183
(392, 32)
0.6862244897959183
(392, 33)
0.6862244897959183
(392, 34)
0.6862244897959183
(392, 35)
0.6811224489795918
(392, 36)
0.6811224489795918
(392, 37)
0.6785714285714286
(392, 38)
0.6785714285714286
(392, 39)
0.6785714285714286
(392, 40)
0.6785714285714286
(392, 41)
0.6811224489795918
(392, 42)
0.6811224489795918
(392, 43)
0.6811224489795918
(392, 44)
0.6836734693877551
(392, 45)
0.6811224489795918
(392, 46)
0.6811224489795918
(392, 47)
0.6836734693877551
(392, 48)
0.6836734693877551
(392, 49)
0.6862244897959183
(392, 50)
0.6836734693877551
0.6862244897959183
(392, 27)
#########################sigmoid, 1#############################
(392, 1)
0.6275510204081632
(392, 2)
0.6147959183673469
(392, 3)
0.6530612244897959
(392, 4)
0.6122448979591837
(392, 5)
0.6173469387755102
(392, 6)
0.6

In [98]:
linear_r2 = [0.8622448979591837,0.8877551020408163,0.8698979591836735,0.8647959183673469,0.8647959183673469]
poly_r2 = [0.7066326530612245,0.6454081632653061,0.6989795918367347,0.6989795918367347,0.7040816326530612]
sigmoid_r2 = [0.6862244897959183,0.6658163265306123,0.6964285714285714,0.6658163265306123,0.6683673469387755]

linear_n = [24,30,25,36,20]
poly_n = [45,7,45,44,47]
sigmoid_n = [27,29,30,31,33]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)


0.8698979591836735 +- 0.008124038404635955
0.6908163265306122 +- 0.02006987792688336
0.6765306122448979 +- 0.010990905331227237
27.0 +- 4.832900578327677
37.6 +- 13.438226549660486
30.0 +- 1.753077294359835


## Supervised PCA

In [100]:
for kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        print(f'#########################{kernel}, {curr_seed}#############################')
        best_score = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel='linear', degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
    

#########################linear, 0#############################
(392, 1)
0.625
(392, 2)
0.6122448979591837
(392, 3)
0.6326530612244898
(392, 4)
0.6505102040816326
(392, 5)
0.7117346938775511
(392, 6)
0.6709183673469388
(392, 7)
0.6862244897959183
(392, 8)
0.7525510204081632
(392, 9)
0.7908163265306123
(392, 10)
0.8137755102040817
(392, 11)
0.7908163265306123
(392, 12)
0.7959183673469388


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 13)
0.7806122448979592


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 14)
0.8010204081632653


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 49)
0.8290816326530612


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(392, 50)
0.826530612244898
0.875
(392, 31)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [101]:
linear_r2 = [0.8698979591836735,0.875,0.8826530612244898,0.8801020408163265,0.9005102040816326]
poly_r2 = [0.8724489795918368,0.8698979591836735,0.8852040816326531,0.8801020408163265,0.8903061224489796]
sigmoid_r2 = [0.8673469387755102,0.9005102040816326,0.8852040816326531,0.8852040816326531,0.875]

linear_n = [38,37,41,44,47]
poly_n = [39,29,22,31,45]
sigmoid_n = [46,45,36,27,31]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)


0.8816326530612244 +- 0.009121403400793086
0.8795918367346939 +- 0.006693280212272581
0.8826530612244898 +- 0.009797958971132715
41.4 +- 3.260912264995794
33.2 +- 7.021069092381872
37.0 +- 6.582799404508692


## Isomap

In [105]:
from sklearn.manifold import Isomap

for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(392, 1)
0.6198979591836735


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(392, 2)
0.6173469387755102
(392, 3)
0.6122448979591837


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(392, 48)
0.6096938775510204


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(392, 49)
0.6020408163265306
(392, 50)
0.5943877551020408
0.6403061224489796
(392, 8)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


In [106]:
best_isomap_r2 = [0.6428571428571429,0.6454081632653061,0.6530612244897959,0.6403061224489796,0.6403061224489796]

best_isomap_n = [37,5,9,4,8]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.6443877551020408 +- 0.004147288270665538
12.6 +- 10.815222457259026


## LLE

In [107]:
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(392, 1)
0.6147959183673469
(392, 2)
0.6301020408163265


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(392, 3)
0.6352040816326531
(392, 4)
0.6275510204081632


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(392, 48)
0.625


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(392, 49)
0.6224489795918368
(392, 50)
0.6173469387755102
0.6556122448979592
(392, 15)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


In [108]:
best_isomap_r2 = [0.6428571428571429,0.6428571428571429,0.6428571428571429,0.6377551020408163,0.6556122448979592]

best_isomap_n = [18,50,16,42,15]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.6443877551020408 +- 0.005215361924162125
28.2 +- 12.958550042346557


## NCA

In [109]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis as NCA

for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = NCA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(392, 1)
0.7755102040816326


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(392, 2)
0.6198979591836735


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(392, 48)
0.8367346938775511


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exampl

(392, 49)
0.8367346938775511
(392, 50)
0.8418367346938775
0.8775510204081632
(392, 21)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [110]:
best_isomap_r2 = [0.8698979591836735,0.8928571428571429,0.8826530612244898,0.8647959183673469,0.8775510204081632]

best_isomap_n = [22,30,26,22,21]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.8775510204081634 +- 0.008602325267042639
24.2 +- 2.9543383692461496
